In [1]:
import os
os.chdir("../src/")

In [2]:
import ipynb
from ipynb.fs.full import international_covid_data as cdi
from ipynb.fs.full import covid_data as cd
import pandas as pd
import numpy as np

In [3]:
def normalize(x, population):
    return (x/population)*10000000

## World Data

#### CASES

In [13]:
#CONVERT FROM TOTAL CASES TO NEW CASES
df = cdi.get_international_cases()
date_column = df['date']
cases = df.loc[:,df.columns[1:]].fillna(method='ffill')
cases = cases.fillna(value=0)
new_cases = cases.diff().fillna(value=0)
new_cases = new_cases.astype(int)
new_cases.insert(loc=0, column="Date", value=date_column)
new_cases = new_cases.loc[1:]
population_df = cdi.get_international_population()
population_df = population_df.rename(columns={"countriesAndTerritories": "country"})
new_cases = new_cases.rename(columns={"United States": "United States of America"})

#REPLACE SPACES WITH UNDERSCORE IN COLUMN NAMES IN CASES DATA
new_cases.columns = new_cases.columns.str.replace(' ','_')
display(new_cases.loc[new_cases["Date"] == "2020-03-03"])
#CALCULATE NORMALIZED VALUES
for country in new_cases.columns[1:]:
    population = population_df[population_df['country'] == country]['population']
    if(population.empty): 
        new_cases = new_cases.drop(country, 1)
        continue
    new_cases[country] = new_cases[country].apply(normalize, args=(population,))
display(new_cases)
new_cases.iloc[:,1:] = new_cases.iloc[:,1:].round(0).astype(int)
    
new_cases.to_csv('../data/Covid_International/new_cases_normalized.csv', mode='w', index = False, header=True)

,Date,World,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua_and_Barbuda,Argentina,...,United_States_Virgin_Islands,Uruguay,Uzbekistan,Vatican,Venezuela,Vietnam,Western_Sahara,Yemen,Zambia,Zimbabwe
63,2020-03-03,1857,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,Date,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua_and_Barbuda,Argentina,Armenia,...,United_States_of_America,United_States_Virgin_Islands,Uruguay,Uzbekistan,Venezuela,Vietnam,Western_Sahara,Yemen,Zambia,Zimbabwe
1,2020-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,2020-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,2020-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,2020-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,2020-01-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2020-10-10,2.568823,580.304399,33.294533,16566.362519,28.905016,0.0,0.0,3340.798854,2072.060458,...,1754.729170,95.764343,71.968810,65.732078,238.078969,0.513671,0.0,0.000000,41.884347,28.931044
285,2020-10-11,0.000000,573.354646,31.014086,0.000000,22.211223,0.0,0.0,2749.814435,2190.174654,...,1639.594139,0.000000,48.938791,164.927759,266.212378,0.205468,0.0,0.335278,20.670197,10.765040
286,2020-10-12,24.660696,583.779276,30.101907,0.000000,65.416615,0.0,0.0,2281.186581,2412.904280,...,1317.119376,0.000000,74.847563,60.951563,240.540643,0.205468,0.0,0.000000,64.730355,0.672815
287,2020-10-13,33.137811,594.203906,34.890846,38697.987446,36.511599,0.0,0.0,2107.276527,3762.780800,...,1258.388728,0.000000,54.696296,130.269027,217.682248,0.102734,0.0,0.335278,49.499683,6.728150


#### DEATHS

In [14]:
#CONVERT FROM TOTAL DEATHS TO NEW DEATHS
df = cdi.get_international_deaths()
date_column = df['date']
deaths = df.loc[:,df.columns[1:]].fillna(method='ffill')
deaths.fillna(value=0)
new_deaths = deaths.diff().fillna(value=0)
new_deaths = new_deaths.astype(int)
new_deaths.insert(loc=0, column="Date", value=date_column)
new_deaths = new_deaths.loc[1:]

#GET POPULATION DATA
population_df = cdi.get_international_population()
population_df = population_df.rename(columns={"countriesAndTerritories": "country"})
new_deaths = new_deaths.rename(columns={"United States": "United States of America"})

#REPLACE SPACES WITH UNDERSCORE IN COLUMN NAMES IN CASES DATA
new_deaths.columns = new_deaths.columns.str.replace(' ','_')

#CALCULATE NORMALIZED VALUES
for country in new_deaths.columns[1:]:
    population = population_df[population_df['country'] == country]['population']
    if(population.empty): 
        new_deaths = new_deaths.drop(country, 1)
        continue
    new_deaths[country] = new_deaths[country].apply(normalize, args=(population,))

new_deaths.iloc[:,1:] = new_deaths.iloc[:,1:].round(0).astype(int)

new_deaths.to_csv('../data/Covid_International/new_deaths_normalized.csv', mode='w', index = False, header=True)

## US Data

#### CASES

In [ ]:
population_county = cd.get_county_population()
population_grouped = population_county.groupby(population_county.State)
population_state = population_grouped.agg(np.sum)

cases_state = pd.read_csv('../data/Covid/Daily/covid_daily_cases.csv')
cases_state = cases_state.rename(columns={"Unnamed: 0": "State"})
cases_state.set_index("State", inplace=True)

states = cases_state.index
for state in states:
    population = population_state.loc[state]["population"]
    cases_state.loc[state] = cases_state.loc[state].apply(normalize, args=(population,))

cases_state = cases_state.round(0).astype(int)

cases_state.to_csv('../data/Covid/Daily/daily_new_cases_normalized.csv', mode='w', index = True, header=True)

#### DEATHS

In [ ]:
population_county = cd.get_county_population()
population_grouped = population_county.groupby(population_county.State)
population_state = population_grouped.agg(np.sum)

deaths_state = pd.read_csv('../data/Covid/Daily/covid_daily_deaths.csv')
deaths_state = deaths_state.rename(columns={"Unnamed: 0": "State"})
deaths_state.set_index("State", inplace=True)

states = deaths_state.index

for state in states:
    population = population_state.loc[state]["population"]
    deaths_state.loc[state] = deaths_state.loc[state].apply(normalize, args=(population,))

deaths_state = deaths_state.round(0).astype(int)

deaths_state.to_csv('../data/Covid/Daily/daily_new_deaths_normalized.csv', mode='w', index = True, header=True)